### Best Management Practice (BMP) Application Programming Interface (API)
###### Academy of Natural Sciences of Drexel University,  Drexel University College of Computing and Informatics

The Drexel University College of Computing and Informatics (CCI) and the Academy of Natural Sciences (ANS) of Drexel University have developed Application Programming Interfaces (APIs) which incorporate novel algorithms to apply efficient solutions to complex environmental queries. The APIs are built in Python using a GeoDjango Web framework, Nginx, Docker, PostgreSQL, and Swagger.

The Best Management Practice (BMP) API returnes BMP specific nutrient and sediment reduction estimates for a user-supplied area of interest. The API supports BMPs within the Delaware and Chesapeake Bay basins. In the Delaware, the API uses the Generalized Watershed Loading Funtion Enhanced (GWLF-E) model outputs at the HUC12 scale. In the Chesapeake, the API relies on the Chesapeake Bay Model scenarios within the Chesapeake Assessment Scenario Tool (CAST), also at the HUC12 scale. The API calculates the watershed boundary and land cover distribution for the BMP area of interest, then re-allocates the loads using BMP specific modeling approaches and efficiencies. BMP efficiencies used vary spatially depending on the watershed. This API is being extended to incorporate local factors that affect pollutant transport and BMP efficiency, such as ecoregion or soils.

In [ ]:
from ipyleaflet import Map, basemaps, Polygon, DrawControl, WidgetControl, GeoJSON, MeasureControl, basemap_to_tiles, SplitMapControl
import ipywidgets as widgets
from ipywidgets import IntSlider, ColorPicker, jslink, Dropdown, interact, FloatSlider
import requests
import geojson
import json
import shapely
from shapely.geometry import mapping, MultiPolygon, Polygon, LineString, Point
import matplotlib.pyplot as plt
import collections
from math import sqrt
import numpy as np
from shapely.ops import unary_union

In [ ]:
# Copy in the lookup table from the BMP API
bmp_lookups = {"": [],
                      "Agricultural Animal": [
                      #"",
                      "Animal Waste Management Systems",
                      "Waste Storage Facility",
                      "Waste Storage Pond",
                      "Waste Storage Structure"
                      ],
                      "Stream Restoration": [
                      #"",
                      "Watering Facility",
                      "Fence",
                      "Non-Urban Stream Restoration",
                      "Stream Channel Stabilization",
                      "Streambank and Shoreline Protection",
                      "Urban Stream Restoration"
                      ],
                    "Land Use Change": [
                      #"",
                      "Tree and Shrub Establishment",
                      "Tree Planting",
                      "Barnyard Runoff Controls",
                      "Conservation Easement",
                      "Heavy Use Area Protection",
                      "Roof Runoff Management",
                      "Roof Runoff Structure",
                      "Roofs and Covers"
                      ],
                    "Agricultural Land": [
                      #"",
                      "Conservation Tillage",
                      "Reduced Tillage",
                      "High Residue Tillage Management",
                      "Soil Conservation and Water Quality Plans",
                      "Comprehensive Nutrient Management Plan",
                      "Nutrient Management",
                      "Conservation Cover",
                      "Cover Crop",
                      "Grazing Land Protection",
                      "Prescribed Grazing"
                      ],
                    "Urban Stormwater Management": [
                      #"",
                      "Constructed Wetland",
                      "Dry Extended Detention Ponds",
                      "Wet Pond",
                      "Wet Ponds & Wetlands",
                      "Bioretention",
                      "Bioretention/raingardens - A/B soils  no underdrain",
                      "Bioretention/raingardens - A/B soils no underdrain",
                      "Bioretention/raingardens - C/D soils  underdrain",
                      "Bioretention/raingardens - C/D soils  no underdrain",
                      "Bioswale",
                      "Dry Well/Seepage Pit",
                      "Impervious Surface Reduction",
                      "Infiltration Practices w/o Sand  Veg. - A/B soils  no underdrain",
                      "Permeable Pavement w/o Sand  Veg. - A/B soils  no underdrain",
                      "Stormwater Performance Standard-Runoff Reduction",
                      "Urban Infiltration Practices",
                      "Stormwater Performance Standard-Stormwater Treatment"
                      ],
                    "Polygon Drainage": [
                      #"",
                      "Forest Buffer",
                      "Forest Buffer - Narrow",
                      "Riparian Herbaceous Cover",
                      "Grass Buffers",
                      "Grass Buffer - Narrow",
                      "Grassed Waterway",
                      "Wetland Creation - Floodplain",
                      "Wetland Restoration",
                      "Wetland Restoration - Floodplain"
                      ],
                    "Exclusion Buffer": [
                      #"",
                      "Forest Buffer-Streamside with Exclusion Fencing",
                      "Forest Buffer-Narrow with Exclusion Fencing",
                      "Grass Buffer-Streamside with Exclusion Fencing",
                      "Grass Buffer-Narrow with Exclusion Fencing",
                      ]
                  }

messages_arch = {"": '',
                 "Agricultural Animal": 'Draw the point where the farm is located and enter the number of animals treated by the BMP.',
                 "Stream Restoration": 'Draw the line representing the BMP and (optionally) enter the length of stream treated.',
                 "Land Use Change": 'Draw the point or polygon representing the BMP and enter the acres treated (optional if a polygon is drawn).',
                 "Agricultural Land": 'Draw the point or polygon representing the BMP and enter the acres treated (optional if a polygon is drawn).',
                 "Urban Stormwater Management": 'Draw the polygon for the BMP or the drainage area to the BMP and provide the drainage area (acres, optional), percent impervious (%, optional) and runoff capture (inches, optional).',
                 "Polygon Drainage": 'Draw the polygon footprint for your BMP.',
                 "Exclusion Buffer": 'Draw the polygon footprint for your BMP and enter the length of fencing.'
                }

# Sort this dictionary for better viewing
bmp_lookups_sort = {}
for arch in sorted(bmp_lookups):
    bmp_lookups_sort[arch] = bmp_lookups[arch]
    temp = []
    for bmp in sorted(bmp_lookups[arch]):
        temp.append(bmp)
        #print("%s: %s" % (key, bmp_lookups[key]))
    bmp_lookups_sort[arch] = temp

bmp_lookups = dict(bmp_lookups_sort)
del(bmp_lookups_sort)

# bmp_lookups.keys()


In [ ]:
# Set up all of the different fields to populate

acres_treated = 0.0
percent_impervious = 0.0
runoff_capture = 1.0
stream_feet_treated = 0.0
number_of_units = 1.0
animal_treated = {
    "chickens_broilers": 0.0,
    "chickens_layers": 0.0,
    "cows_beef": 0.0,
    "cows_dairy": 0.0,
    "horses": 0.0,
    "pigs_hogs_swine": 0.0,
    "sheep": 0.0,
    "turkeys": 0.0
    }

acres_treated_text = widgets.BoundedFloatText(
    value=acres_treated,
    min=0.00,
    max=10000.0,
    placeholder='Enter Acres Treated:',
    description='Acres',
    disabled=False
)

drainage_acres_treated_text = widgets.BoundedFloatText(
    value=acres_treated,
    min=0.00,
    max=10000.0,
    placeholder='Enter Drainage Area Treated (Ac.):',
    description='DA (Ac.)',
    disabled=False
)

percent_impervious_text = widgets.BoundedFloatText(
    value=percent_impervious,
    placeholder='Enter Percent Impervious:',
    min=0.00,
    max=100.0,
    description='% Impervious',
    disabled=False
)

runoff_capture_text = widgets.BoundedFloatText(
    value=runoff_capture,
    min=0.05,
    max=2.5,
    step=0.1,
    placeholder='Enter Runoff Capture (inches):',
    description='Capture (in.)',
    disabled=False
)

runoff_capture_slider = FloatSlider(min=0.05, max=2.5, step=0.05, value=1)

stream_feet_treated_text = widgets.BoundedFloatText(
    value=stream_feet_treated,
    min=0.00,
    max=10000.0,
    placeholder='Enter Stream Feet Treated:',
    description='',
    disabled=False
)

number_of_units_text = widgets.BoundedFloatText(
    value=number_of_units,
    min=0.00,
    max=10.0,
    placeholder='Enter Number of Units:',
    description='',
    disabled=False
)

# FOR ANIMALS

chickens_broilers_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Broilers',
    disabled=False
)

chickens_layers_text = widgets.BoundedFloatText(
    value = None,
    min=0.00,
    max=1000.0,
    description='Layers',
    disabled=False
)

cows_beef_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Beef Cows',
    disabled=False
)

cows_dairy_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Dairy Cows',
    disabled=False
)

horses_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Horses',
    disabled=False
)

pigs_hogs_swine_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Hogs/Swine',
    disabled=False
)

sheep_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Sheep',
    disabled=False
)

turkeys_text = widgets.BoundedFloatText(
    value=0,
    min=0.00,
    max=1000.0,
    description='Turkeys',
    disabled=False
)

In [ ]:
# Put in a DropDown Widget to Select a BMP and enter the necessary information
bmps_selected = []
arches_selected = []


def select_bmp(Archetype):
    arches_selected.append(Archetype)
    bmpW.options = bmp_lookups[Archetype]

def print_bmp(BMP):
    if BMP == None:
        pass
    else:
        print('Selected BMP: {}\n'.format(BMP))
        print(messages_arch[str(scW.value)])
        bmps_selected.append(BMP)
        if str(scW.value) == 'Agricultural Animal':
            display(chickens_broilers_text)
            display(chickens_layers_text)
            display(cows_beef_text)
            display(cows_dairy_text)
            display(horses_text)
            display(pigs_hogs_swine_text)
            display(sheep_text)
            display(turkeys_text)
        elif str(scW.value) == 'Agricultural Land':
            display(acres_treated_text)
        elif str(scW.value) == 'Exclusion Buffer':
            display(stream_feet_treated_text)
        elif str(scW.value) == 'Land Use Change':
            display(acres_treated_text)
        elif str(scW.value) == 'Polygon Drainage':
            pass
        elif str(scW.value) == 'Stream Restoration':
            display(stream_feet_treated_text)
        elif str(scW.value) == 'Urban Stormwater Management':
            display(drainage_acres_treated_text)
            display(percent_impervious_text)
            display(runoff_capture_text)
            
        
scW = widgets.Dropdown(options=bmp_lookups.keys())
init = scW.value
bmpW = widgets.Dropdown(options=bmp_lookups[init])
disp_bmp = widgets.interactive(print_bmp, BMP=bmpW)
disp_arch = widgets.interactive(select_bmp, Archetype=scW)
display(disp_arch)
display(disp_bmp)


# Create the map
# Map centred on Wissahickon Park
displaymap1 = Map(center = (40.050, -75.215), zoom = 12, min_zoom = 1, max_zoom = 20,  scroll_wheel_zoom = True,
                 basemap = basemaps.Esri.WorldStreetMap)

# Enable user to draw polygons and lines
draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 4,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.3
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": True,
}

# Save the drawn geometry to a geojson
feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    # Save the GeoJSON when it's drawn on the map  
    feature_collection['features'].append(geo_json)

# Add in a zoom slider bar for fun
zoom_slider1 = IntSlider(description='Zoom level:', min=0, max=20, value=12)
jslink((zoom_slider1, 'value'), (displaymap1, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider1, position='topright')

# Add in ability to do measurements
measure = MeasureControl(
    position='bottomleft',
    active_color = 'red',
    primary_length_unit = 'feet'
)

displaymap1.add_control(measure)

measure.completed_color = 'red'

measure.add_length_unit('miles', 0.000621371, 1)
measure.secondary_length_unit = 'miles'

measure.add_area_unit('sqkm', 0.0000001, 1)
measure.secondary_area_unit = 'sqkm'

# Add everything into the map
draw_control.on_draw(handle_draw)
displaymap1.add_control(draw_control)

left_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
right_layer = basemap_to_tiles(basemaps.Esri.WorldStreetMap)

#split_control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
#displaymap1.add_control(split_control)

# Show the map
displaymap1

In [ ]:
# Show the BMP
bmp = bmps_selected[len(bmps_selected)-1]
archetype = arches_selected[len(arches_selected)-1]
print('{}: {}'.format(archetype, bmp))

# Show the last drawn shape
last_geom_idx = len(feature_collection['features']) - 1
last_geom = feature_collection['features'][last_geom_idx]['geometry']
print(last_geom)

# Update the paramters with the user supplied input
acres_treated = drainage_acres_treated_text.value
percent_impervious = percent_impervious_text.value
runoff_capture = runoff_capture_text.value
stream_feet_treated = stream_feet_treated_text.value
number_of_units = number_of_units_text.value
animal_treated = {
    "chickens_broilers": chickens_broilers_text.value,
    "chickens_layers": chickens_layers_text.value,
    "cows_beef": cows_beef_text.value,
    "cows_dairy": cows_dairy_text.value,
    "horses": horses_text.value,
    "pigs_hogs_swine": pigs_hogs_swine_text.value,
    "sheep": sheep_text.value,
    "turkeys": turkeys_text.value
    }


#### Call BMP API

In [ ]:
# Call the Watersheds API with the drawn shape
_x = dict(eval(str(last_geom)))
_payload = {"bmp_geometry": _x,
            "bmp_type": bmp,
            "bmp_group": archetype,
            "animal_treated": animal_treated,
            "acres_treated": acres_treated,
            "stream_feet_treated": stream_feet_treated,
            "percent_impervious": percent_impervious,
            "runoff_capture": runoff_capture,
            "number_of_units": number_of_units,
            "model": "gwlfe"
           }
_payload = json.dumps(_payload)
_url = 'https://watersheds-staging.cci.drexel.edu/api/bmp/'
_headers = {'Content-Type': 'application/json'}
_r_bmp = requests.post(_url, data = _payload , headers= _headers, allow_redirects=True, verify=True)

print('{')
for k, v in eval(_payload).items():
    print('\t{}: {}'.format(k, v))
print('}')
# print(_r_bmp.text)

In [ ]:
# Cache what we want to use later in the visualization

# Save the watershed as a geojson and a shapely geometry object
return_dict = eval(str(json.loads(_r_bmp.text)))

watershed_shapely = shapely.geometry.shape(return_dict['watershed_geometry'])
if watershed_shapely.type == 'Polygon':
    watershed_shapely = MultiPolygon([watershed_shapely])
watershed_geojson = geojson.loads(geojson.dumps(mapping(watershed_shapely)))

#watershed_shapely = unary_union(watershed_shapely)
#watershed_geojson = geojson.loads(geojson.dumps(mapping(watershed_shapely)))

# Cache land cover and final recutions
lulcs = return_dict['land_cover_acres']
reductions = return_dict['reduction_lbyr']
p_impervious = return_dict['percent_impervious']
lu_total = 0
for lulc, acre in lulcs.items():
    lu_total += acre
print(lu_total)
print('Input area ammended land cover acres: {}\n'.format(lulcs))

# In case we get a zero area, double check by re-sending the watershed boundary
# Reset area for providing warnings with urban stormwater management BMPs
try:
    if lu_total >= 0:
        lulcs = {}
        lu_total = 0
        n = 1
        # Call the Fast Zonal API with the Watershed Boundary
        for geom in watershed_shapely:
            print(n, end='')
            m = MultiPolygon([geom])
            new_watershed_geojson = geojson.loads(geojson.dumps(mapping(m)))
            _x = json.dumps(new_watershed_geojson)
            _url = 'https://watersheds.cci.drexel.edu/api/fzs/'
            _headers = {}
            _r = requests.post(_url, data = _x , headers= _headers, allow_redirects=True, verify=True)
            lulcs_loop = eval(_r.text)
            for lulc, sqm in lulcs_loop.items():
                if lulc in lulcs.keys():
                    _v = sqm / 4046.86
                    lulcs[lulc] += _v
                else:
                    lulcs[lulc] = sqm / 4046.86
            n += 1
        for lulc, acre in lulcs.items():
            lu_total += acre
        
    print('\nRaw land cover acres: {}'.format(lulcs))
    print(lu_total)
except Exception as e:
    print(e)

#### Call OSI API

In [ ]:
try:
    n = 1
    pwr = {}
    for geom in watershed_shapely:
        print(n, end='')
        _x = json.dumps(shapely.geometry.mapping(geom))
        _url = 'https://watersheds.cci.drexel.edu/api/osigeo/'
        _headers = {}
        _r_osi = requests.post(_url, data = _x , headers= _headers, allow_redirects=True, verify=True)
        pwr_loop = eval(_r_osi.text)
        # Convert to acres / feet
        for r, sqm in pwr_loop.items():                
            if r in pwr.keys():
                if r == 'str_bank':
                    _v = sqm / 1609.34
                else:
                    _v = sqm / 4046.86
                
                pwr[r] += _v
            else:
                if r == 'str_bank':
                    pwr[r] = sqm / 1609.34
                else:
                    pwr[r] = sqm / 4046.86
        n += 1

    # Get PWR percents
    pwr_percent = {}
    for r, acres in pwr.items():
        try:
            value = acres / lu_total * 100
            if value >= 100:
                value = 100
            pwr_percent[r] = value
        except:
            pwr_percent[r] = 0.001
    print('\nValues: {}'.format(pwr))
    print('Percents: {}'.format(pwr_percent))
        
except Exception as e:
    print(e)
    print('test')

### Create the final visualization

In [ ]:
# Generate the second map
# Show the polygon and its watershed on a new map
try:
    zoom_level = round((-0.697 * np.log(lu_total)) + 18.003)
except:
    zoom_level = 15.0

displaymap2 = Map(center = (watershed_shapely.centroid.coords[0][1], watershed_shapely.centroid.coords[0][0])
                 , min_zoom = 1, max_zoom = 20, scroll_wheel_zoom = True,
    basemap=basemaps.Esri.WorldStreetMap)

zoom_slider2 = IntSlider(description='Zoom level:', min=0, max=20, value=zoom_level)
jslink((zoom_slider2, 'value'), (displaymap2, 'zoom'))
widget_control2 = WidgetControl(widget=zoom_slider2, position='topright')

watershed = GeoJSON(
    data=watershed_geojson,
    style={
        'opacity': 1, 'dashArray': '2', 'fillOpacity': 0.2, 'weight': 1, 'color': 'blue'
    }
)

polygon = GeoJSON(
    data=last_geom,
    style={
        'opacity': 1, 'fillOpacity': 0.1, 'weight': 2, 'color': 'green'
    }
)

measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)

displaymap2.add_control(measure)

measure.completed_color = 'red'

measure.add_length_unit('yards', 1.09361, 4)
measure.secondary_length_unit = 'yards'

measure.add_area_unit('sqyards', 1.19599, 4)
measure.secondary_area_unit = 'sqyards'

displaymap2.add_layer(polygon)
displaymap2.add_layer(watershed)
displaymap2.add_control(widget_control2)

left_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
right_layer = basemap_to_tiles(basemaps.Esri.WorldStreetMap)

split_control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
displaymap2.add_control(split_control)

# displaymap2


In [ ]:
############################
# FINAL VISUALIZATION PAGE #
############################

# Show land cover summary
print('Total Drainage Area Acres: {}'.format(round(lu_total,2)))
# for lulc, sqm in lulcs.items():
#     print('{}: {}'.format(lulc, round(sqm,2)), end =", ")

print('Percent Impervious: {}%'.format(round(p_impervious, 2)))

# Show PWR summary, use the wetlands from the landcover call
pwr_labels = {'str_bank': 'Stream Bank (mi)', 'head_pwr': 'Headwaters (ac)', 'ara_pwr': 'Active River Area (ac)'
              , 'wet_pwr': 'Wetlands (ac)', 'tot_pwr': 'Total Priority Water Resources (PWR) (ac)'}
wetlands_acres = 0
for lulc, acre in lulcs.items():
    if lulc == '90' or lulc == '95':
        wetlands_acres += acre

try:
    wetlands_percent = wetlands_acres / lu_total * 100
except:
    print("Total Area Returned Zero")

for r, sqm in pwr.items():
    if r == 'wet_pwr':
        print('{}: {}'.format(pwr_labels[r], round(wetlands_acres,2)))
    else:
        print('{}: {}'.format(pwr_labels[r], round(sqm,2)))

###############################
# Pie charts for the PWR call #
###############################

labels_head = 'Headwaters', ''
sizes_head = [pwr_percent['head_pwr'], 100 - pwr_percent['head_pwr']]

labels_ara = 'ARA', ''
sizes_ara = [pwr_percent['ara_pwr'], 100 - pwr_percent['ara_pwr']]

labels_wet = 'Wetlands', ''
sizes_wet = [wetlands_percent, 100 - wetlands_percent]

labels_pwr = 'Priority\nWater Resources', ''
sizes_pwr = [pwr_percent['tot_pwr'], 100 - pwr_percent['tot_pwr']]

explode = (0.1, 0.0)  # only "explode" the 1st slice
pie_colors = ["lightblue", "lightgrey"]

fig1, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
fig1.set_size_inches(15,15)

ax1.pie(sizes_head, explode=explode, labels=labels_head, autopct='%1.1f%%',
        shadow=True, startangle=90, colors = pie_colors, textprops={'fontsize': 12})

ax2.pie(sizes_ara, explode=explode, labels=labels_ara, autopct='%1.1f%%',
        shadow=True, startangle=90, colors = pie_colors, textprops={'fontsize': 12})

ax3.pie(sizes_wet, explode=explode, labels=labels_wet, autopct='%1.1f%%',
        shadow=True, startangle=90, colors = pie_colors, textprops={'fontsize': 12})

ax4.pie(sizes_pwr, explode=explode, labels=labels_pwr, autopct='%1.1f%%',
        shadow=True, startangle=90, colors = pie_colors, textprops={'fontsize': 12})

plt.show()

##########################
# Create Land Cover Plot #
##########################

plot_length = int(len(lulcs))
if plot_length > 10:
    plot_length = 10

fig2 = plt.figure()
fig2.set_size_inches(plot_length,5)
ax2 = fig2.add_axes([0,0,1,1])

colormap = {'11': '#5475a8','21': '#e8d1d1','22': '#e29e8c','23': '#ff0000','24': '#b50000','31': '#d2cdc0',
            '41': '#86c77e','42': '#39814e','43': '#d5e7b0','52': '#dcca8f','71': '#fde9aa','81': '#fbf65c',
            '82': '#ca9146','90': '#c8e6f8','95': '#64b3d5'}

crosswalk = {'11': 'Open Water',
             '21': 'Developed Open Space',
             '22': 'Developed Low Intensity',
             '23': 'Developed Medium Intensity',
             '24': 'Developed High Intensity',
             '31': 'Barren Land',
             '41': 'Deciduous Forest',
             '42': 'Evergreen Forest',
             '43': 'Mixed Forest',
             '52': 'Shrub/Scrub',
             '71': 'Grassland Herbaceous',
             '81': 'Pasture Hay',
             '82': 'Cultivated Crops',
             '90': 'Woody Wetlands',
             '95': 'Emergent Herbaceous Wetlands'}

colors = {x: colormap[x] for x in lulcs}
x_labels = {x: crosswalk[x] for x in lulcs}

ax2.bar(list(lulcs.keys()), list(lulcs.values()), align='center', color=colors.values())
#ax.set_xticklabels(x_labels.values())
ax2.set_ylabel('Area (acres)')
ax2.set_title('Watershed Land Cover Distribution (NLCD 2011)')
handles = [plt.Rectangle((0,0),len(lulcs),len(lulcs), color=colors[label]) for label in x_labels]
ax2.legend(handles, x_labels.values(), bbox_to_anchor=(1.02, 1), loc='upper left')

plt.show()

##########################
# Show Reductions #
##########################

if reductions['tss'] <= 2000.0:
    print('The %s is estimated to reduce:\n\tNitrogen: %.2f pounds\n'
    '\tPhosphorus: %.2f pounds\n'
    '\tSediment: %.2f pounds' % (bmp, reductions['tn'],reductions['tp'],reductions['tss']))
else:
    print('The %s is estimated to reduce:\n\tNitrogen: %.2f pounds\n'
    '\tPhosphorus: %.2f pounds\n'
    '\tSediment: %.2f tons' % (bmp, reductions['tn'],reductions['tp'],reductions['tss']/2000.0))

if archetype == 'Urban Stormwater Management':
    if acres_treated > lu_total:
        print('Warning: Number of sewershed acres treated given (%.2f) is larger than the expected overland drainage area (%.2f).'%(acres_treated, lu_total))
    elif 0 < acres_treated < lu_total:
        print('Warning: Number of sewershed acres treated given (%.2f) is smaller than the expected overland drainage area (%.2f). This is to be expected with engineered urban stormwater networks.'%(acres_treated, lu_total))
elif archetype == 'Polygon Drainage' or archetype == 'Exclusion Buffer':
    if 'Buffer' in bmp and lu_total > 100.0:
        print("Warning: The buffer drains a large area (%.2f acres) and may contain concentrated flow paths, which would reduce estimated reduction."%(lu_total))
        
################
# Show the map #
################

displaymap2

###### Scratch Space (save outputs temporarily)


testing

In [ ]:
The Bioretention is estimated to reduce:
	Nitrogen: 0.32 pounds
	Phosphorus: 0.11 pounds
	Sediment: 451.17 pounds
        
The Bioretention is estimated to reduce:
	Nitrogen: 0.43 pounds
	Phosphorus: 0.15 pounds
	Sediment: 594.88 pounds